In [1]:
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pl.read_csv('../sharechat_recsys2023_data/train/*.csv', separator='\t', infer_schema_length=10000)
df.head()

f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,…,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
2541162,51,6581,22294,5491,25604,14597,27941,18800,31372,22970,32266,11402,17705,26082,19819,14709,28694,5774,25952,21553,0,0,0,0,0,0,0,0,0,null,null,3,1,1,1,1,…,0.0,0.0,0.0,0.0,0.0,0.571121,2.510696,0.038564,0.0,0.617025,0.809845,0.077128,2.545229,0.803333,3.397252,0.0,0.0,1.680896,2.841769,69350014,1462.240096,1.599715,7.851265,0.0,0.0,2.82868,1.142243,3.99785,1.713364,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2541260,49,20095,563,22861,15908,22651,27941,21218,869,19343,12213,30670,4230,20899,13481,25546,3249,12020,26758,4222,4625,0,0,0,0,0,0,0,0,null,null,3,0,0,0,1,…,0.0,0.0,0.0,0.0,0.0,0.0,30.726136,0.0,0.0,0.0,0.308513,0.0,0.385641,1.518634,3.205653,0.0,0.0,0.0,0.0,126347712,1639.124417,0.000873,8.157427,1.089364,1.646756,0.0,0.0,0.0,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2541318,51,20095,563,22861,25604,22651,27941,21218,31372,22970,11407,17133,18614,26082,32590,25546,29718,21865,26758,4222,4625,0,0,0,0,0,0,0,0,null,null,3,0,0,0,1,…,0.0,0.0,0.0,0.0,0.0,0.0,25.668477,0.0,0.0,0.0,0.0,0.0,0.0,1.511121,2.413887,0.0,0.295702,0.387899,0.0,192447264,1808.083683,0.000156,8.269833,1.10626,2.220819,0.0,0.0,0.0,0.0,0.038564,1.156922,0.269948,0.0,0.0,0.0,0,0
2541770,64,13658,22294,7003,25604,29975,27941,21218,21533,19343,28360,8659,24696,20899,7057,18162,11918,12020,29982,4740,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,1,…,0.0,0.0,0.0,0.0,0.0,0.0,1.092316,0.038564,0.038564,0.115692,0.038564,0.038564,0.038564,1.336304,1.302389,0.0,0.295702,0.0,4.973095,17889246,110.105037,2.276683,9.171566,0.0,0.0,1.598379,0.0,1.713364,1.142243,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2542002,62,20095,563,31686,15908,590,27941,18800,23218,22970,11407,30670,24696,27291,12126,25546,3249,5774,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,…,0.0,0.0,0.0,0.0,0.0,0.0,17.621443,0.0,0.0,0.0,0.0,0.0,0.0,1.620839,2.184986,0.0,0.0,0.0,0.355221,177032136,1478.281323,0.000615,8.126352,0.200315,0.444261,0.0,0.0,0.0,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0


In [3]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder

In [4]:
from numba import njit


class ColumnCatEncoder:
    
    def __init__(self, name):
        self.col = name
    
    def __call__(self, df):
        if not hasattr(self, 'map_dict'):
            self.map_dict = {}
            vals = df[self.col].value_counts().sort('counts')
            for i, (v, _) in enumerate(vals.iter_rows()):
                self.map_dict[v] = i
            if None not in self.map_dict:
                self.map_dict[None] = i
        n_df = df.with_columns(pl.col(self.col).\
                               map_dict(self.map_dict, \
                                        default=self.map_dict[None]))
        return n_df
    
    
class ColumnCatEncoderChance:
    
    def __init__(self, name):
        self.col = name
    
    def __call__(self, df):
        if not hasattr(self, 'map_click'):
            self.map_click = {}
            self.map_ins = {}
            vals = df[[self.col, 'is_clicked', 'is_installed']].\
                    groupby(self.col).\
                    agg([pl.mean('is_clicked'), pl.mean('is_installed'), pl.count()]).\
                    sort('count')
            for v, click, ins, _ in vals.iter_rows():
                self.map_click[v] = click
                self.map_ins[v] = ins
            if None not in self.map_click:
                self.map_click[None] = self.map_click[v]
                self.map_ins[None] = self.map_ins[v]
        df = df.with_columns(pl.col(self.col).\
                             map_dict(self.map_click, default=self.map_click[None]).\
                             alias(f'{self.col}_click'))
        df = df.with_columns(pl.col(self.col)\
                             .map_dict(self.map_ins, default=self.map_ins[None]).\
                             alias(f'{self.col}_inst'))
        df = df.drop(self.col)
        return df
## TODO FAST coso        

class LogBaseEncoder:
    
    def __init__(self, col):
        self.col = col
        
    def __call__(self, df):
        col = df[self.col].fill_null(strategy='mean').to_numpy()
        if not hasattr(self, 'mean'):
            self.mean = np.mean(col)
            self.std = np.std(col)
            # print(self.mean)
            # print(self.std)
        res = (col-self.mean) / (3 * self.std)
        res = np.where(np.abs(res) <= 1, res, np.sign(res) * (np.log2(np.abs(res)) + 1))
        df = df.with_columns(pl.Series(name=self.col, values=res))
        return df
# steps = [
#     FunctionTransformer(ColumnCatEncoder('f_30'))
# ]


steps = [FunctionTransformer(ColumnCatEncoder(f'f_{c}') 
                             if len(df[f'f_{c}'].value_counts()) > 20 
                             else ColumnCatEncoderChance(f'f_{c}')) 
         for c in range(2, 42)]

steps += [FunctionTransformer(LogBaseEncoder(f'f_{c}'))
         for c in range(42, 80)]

# steps += [FunctionTransformer(ColumnCatEncoder2(f'f_{c}'))
#          for c in range(42, 80)]

# types = {f'f_{c}': 'cat' for c in range(2, 42) if len(df[f'f_{c}'].value_counts()) > 10 }
# for c in range(2, 43):
#     if f'f_{c}' not in types:
#         types[f'f_{c}_click'] = 'prob'
#         types[f'f_{c}_inst'] = 'prob'
# for c in range(42, 80):
#     types[f'f_{c}'] = 'num'
    
    
pipe = make_pipeline(*steps)

In [5]:
%%time
pipe.fit(df)

CPU times: total: 4.64 s
Wall time: 10.7 s


Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<__main__.ColumnCatEncoder object at 0x00000211474FE3E0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<__main__.ColumnCatEncoderChance object at 0x000002114754A170>)),
                ('functiontransformer-3',
                 FunctionTransformer(func=<__main__.ColumnCatEncoder object at 0x000002114754A200>)),
                ('function...
                 FunctionTransformer(func=<__main__.ColumnCatEncoderChance object at 0x000002114754AFB0>)),
                ('functiontransformer-28',
                 FunctionTransformer(func=<__main__.ColumnCatEncoderChance object at 0x000002114754B040>)),
                ('functiontransformer-29',
                 FunctionTransformer(func=<__main__.ColumnCatEncoderChance object at 0x000002114754B0D0>)),
                ('functiontransformer-30',
                 FunctionTransformer(func=<__main__.ColumnCatEncoderChance object at 0x000002114754B160>)), ...])

In [6]:
%%time
train = pipe.transform(df)

CPU times: total: 1.45 s
Wall time: 6.72 s


In [7]:
train.head()

f_0,f_1,f_2,f_4,f_6,f_11,f_12,f_13,f_15,f_17,f_18,f_20,f_21,f_22,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,…,f_23_inst,f_24_click,f_24_inst,f_25_click,f_25_inst,f_26_click,f_26_inst,f_27_click,f_27_inst,f_28_click,f_28_inst,f_29_click,f_29_inst,f_30_click,f_30_inst,f_31_click,f_31_inst,f_32_click,f_32_inst,f_33_click,f_33_inst,f_34_click,f_34_inst,f_35_click,f_35_inst,f_36_click,f_36_inst,f_37_click,f_37_inst,f_38_click,f_38_inst,f_39_click,f_39_inst,f_40_click,f_40_inst,f_41_click,f_41_inst
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2541162,51,125,615,5161,9,20,327,5789,43,899,45,33,23,0.056146,0.115488,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,0.989237,-0.282259,-0.050542,-0.104387,0.653084,0.25541,-0.087799,0.684768,-0.346807,-0.05513,-0.00685,-0.10213,0.028966,0.136635,-0.023027,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.148095,0.113303,0.148095,0.113303,0.221899,0.156893,0.315789,0.182771,0.245159,0.184967,0.277864,0.188139,0.210722,0.170611,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
2541260,49,135,632,5164,19,24,328,5500,48,900,54,32,23,-0.083989,-0.405498,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,0.839139,-0.125328,-0.104387,-0.14403,-0.021474,-0.170255,-0.082985,0.010644,-0.055284,-0.00685,-0.10213,-0.068463,-0.097453,-0.022173,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.148095,0.113303,0.148095,0.113303,0.221899,0.156893,0.213572,0.173447,0.1919,0.16194,0.20007,0.169209,0.210722,0.170611,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
2541318,51,135,632,5164,22,22,325,5395,46,898,54,32,23,-0.100948,-0.405819,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,0.638127,-0.125328,-0.104387,-0.14403,-0.191865,-0.170255,-0.220084,0.00689,-0.05592,-0.00685,-0.075478,-0.045979,-0.097453,-0.021183,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.148095,0.113303,0.148095,0.113303,0.221899,0.156893,0.213572,0.173447,0.1919,0.16194,0.20007,0.169209,0.210722,0.170611,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
2541770,64,115,600,4618,21,23,324,5631,47,900,53,33,23,0.077866,0.52032,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,-0.338632,-0.050542,0.033247,0.005429,-0.170566,-0.129027,-0.206374,-0.08047,-0.056814,-0.00685,-0.075478,-0.068463,0.312201,-0.023797,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.284952,0.229542,0.285276,0.229644,0.21026,0.200825,0.213572,0.173447,0.245159,0.184967,0.277864,0.188139,0.210722,0.170611,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
2542002,62,135,631,5166,22,24,324,5330,48,899,54,32,23,-0.089047,-0.405613,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,0.318304,-0.125328,-0.104387,-0.14403,-0.191865,-0.170255,-0.220084,0.061718,-0.056104,-0.00685,-0.10213,-0.068463,-0.068192,-0.021414,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.284952,0.229542,0.285276,0.229644,0.221899,0.156893,0.213572,0.173447,0.1919,0.16194,0.20007,0.169209,0.210722,0.170611,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167


In [8]:
df.head()

f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,…,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
2541162,51,6581,22294,5491,25604,14597,27941,18800,31372,22970,32266,11402,17705,26082,19819,14709,28694,5774,25952,21553,0,0,0,0,0,0,0,0,0,null,null,3,1,1,1,1,…,0.0,0.0,0.0,0.0,0.0,0.571121,2.510696,0.038564,0.0,0.617025,0.809845,0.077128,2.545229,0.803333,3.397252,0.0,0.0,1.680896,2.841769,69350014,1462.240096,1.599715,7.851265,0.0,0.0,2.82868,1.142243,3.99785,1.713364,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2541260,49,20095,563,22861,15908,22651,27941,21218,869,19343,12213,30670,4230,20899,13481,25546,3249,12020,26758,4222,4625,0,0,0,0,0,0,0,0,null,null,3,0,0,0,1,…,0.0,0.0,0.0,0.0,0.0,0.0,30.726136,0.0,0.0,0.0,0.308513,0.0,0.385641,1.518634,3.205653,0.0,0.0,0.0,0.0,126347712,1639.124417,0.000873,8.157427,1.089364,1.646756,0.0,0.0,0.0,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2541318,51,20095,563,22861,25604,22651,27941,21218,31372,22970,11407,17133,18614,26082,32590,25546,29718,21865,26758,4222,4625,0,0,0,0,0,0,0,0,null,null,3,0,0,0,1,…,0.0,0.0,0.0,0.0,0.0,0.0,25.668477,0.0,0.0,0.0,0.0,0.0,0.0,1.511121,2.413887,0.0,0.295702,0.387899,0.0,192447264,1808.083683,0.000156,8.269833,1.10626,2.220819,0.0,0.0,0.0,0.0,0.038564,1.156922,0.269948,0.0,0.0,0.0,0,0
2541770,64,13658,22294,7003,25604,29975,27941,21218,21533,19343,28360,8659,24696,20899,7057,18162,11918,12020,29982,4740,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,1,…,0.0,0.0,0.0,0.0,0.0,0.0,1.092316,0.038564,0.038564,0.115692,0.038564,0.038564,0.038564,1.336304,1.302389,0.0,0.295702,0.0,4.973095,17889246,110.105037,2.276683,9.171566,0.0,0.0,1.598379,0.0,1.713364,1.142243,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2542002,62,20095,563,31686,15908,590,27941,18800,23218,22970,11407,30670,24696,27291,12126,25546,3249,5774,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,…,0.0,0.0,0.0,0.0,0.0,0.0,17.621443,0.0,0.0,0.0,0.0,0.0,0.0,1.620839,2.184986,0.0,0.0,0.0,0.355221,177032136,1478.281323,0.000615,8.126352,0.200315,0.444261,0.0,0.0,0.0,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0


In [9]:
test = pl.read_csv('../sharechat_recsys2023_data/test/000000000000.csv', separator='\t', infer_schema_length=10000)

In [10]:
test.head()

f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,…,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
64505,67,26325,7152,21563,19475,31440,27941,21621,14659,21574,31460,4473,18614,21302,22947,25546,13234,21865,16851,30153,0,0,0,0,0,0,0,0,0,null,null,3,0,0,0,0,…,1.541848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.029091,0.0,0.0,0.0,0.0,0.0,0.0,1.401956,1.708743,0.0,0.0,0.0,0.0,345640813,1913.393356,1.546994,8.063385,3.836208,10.879233,1.519085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64506,67,20095,563,22861,19475,21280,27941,19203,14659,21574,27957,11402,31705,20899,24789,14709,28694,5774,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,…,0.000119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.292318,0.0,0.0,0.0,0.269948,0.0,0.269948,1.639831,2.206573,0.0,1.62636,1.034398,0.0,146811615,2656.135697,0.000099,8.229785,0.294885,0.647602,0.0,0.0,0.0,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
64507,67,890,22294,18294,21545,20210,27941,18800,9638,19343,30375,8659,17705,28097,38,28225,24285,5774,29982,4740,0,0,0,0,0,0,0,0,0,null,null,0,0,0,0,0,…,0.968559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.253473,0.038564,0.038564,0.038564,0.0,0.0,0.0,1.209622,0.938098,0.0,2.217763,0.0,0.0,53502547,1686.149711,0.990849,7.772551,0.0,0.0,0.982995,0.0,0.0,0.0,0.0,1.156922,0.269948,0.0,0.0,0.0
64508,67,20095,563,22861,25604,21280,27941,18800,14659,22970,31863,8659,17705,26082,9816,18162,24285,12020,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,…,0.000119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.292318,0.0,0.0,0.0,0.617025,0.462769,0.617025,1.639831,2.206573,0.0,0.147851,0.258599,0.0,146811615,2656.135697,0.000099,8.229785,0.294885,0.647602,0.0,0.0,2.284486,0.0,0.0,1.156922,0.269948,0.0,0.0,0.0
64509,67,27426,22294,11338,19475,23855,27941,21218,9638,21574,30778,20023,18614,20496,7057,14709,30304,21865,29982,4740,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,…,1.333393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.074814,0.19282,0.19282,0.347077,0.038564,0.038564,0.115692,1.071687,0.390892,0.0,1.182807,0.0,0.0,60916858,914.246735,1.275921,7.669892,2.116464,26.237742,0.98604,0.0,0.0,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [11]:
test = pipe.transform(test)

In [12]:
test.head()

f_0,f_1,f_2,f_4,f_6,f_11,f_12,f_13,f_15,f_17,f_18,f_20,f_21,f_22,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,…,f_23_inst,f_24_click,f_24_inst,f_25_click,f_25_inst,f_26_click,f_26_inst,f_27_click,f_27_inst,f_28_click,f_28_inst,f_29_click,f_29_inst,f_30_click,f_30_inst,f_31_click,f_31_inst,f_32_click,f_32_inst,f_33_click,f_33_inst,f_34_click,f_34_inst,f_35_click,f_35_inst,f_36_click,f_36_inst,f_37_click,f_37_inst,f_38_click,f_38_inst,f_39_click,f_39_inst,f_40_click,f_40_inst,f_41_click,f_41_inst
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
64505,67,131,607,5166,8,25,325,5153,35,898,49,33,23,-0.145576,0.168262,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,0.096041,-0.125328,-0.104387,-0.14403,-0.191865,-0.170255,-0.220084,-0.047662,-0.056487,-0.00685,-0.10213,-0.068463,-0.097453,-0.018888,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.148095,0.113303,0.148095,0.113303,0.221899,0.156893,0.213572,0.173447,0.1919,0.16194,0.20007,0.169209,0.234142,0.179367,0.220763,0.121653,0.243987,0.154148,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
64506,67,135,632,5165,20,20,286,3050,43,899,54,32,23,-0.055724,-0.405844,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,0.305224,-0.125328,-0.104387,-0.14403,-0.042773,-0.170255,-0.124114,0.071209,-0.056087,-0.00685,0.044459,-0.008506,-0.097453,-0.021866,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.284952,0.229542,0.285276,0.229644,0.21026,0.200825,0.213572,0.173447,0.1919,0.16194,0.20007,0.169209,0.210722,0.170611,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
64507,67,123,590,5132,0,23,327,5109,42,899,53,33,23,-0.12951,-0.045218,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,-0.332227,-0.050542,0.033247,-0.09421,-0.191865,-0.170255,-0.220084,-0.143776,-0.057107,-0.00685,0.097763,-0.068463,-0.097453,-0.023264,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.148095,0.113303,0.148095,0.113303,0.235856,0.170472,0.213572,0.173447,0.1919,0.16194,0.20007,0.169209,0.234142,0.179367,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
64508,67,135,632,5165,11,23,327,5431,42,900,54,32,23,-0.023888,-0.405844,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,0.305224,-0.125328,-0.104387,-0.14403,0.148916,0.324481,-0.000726,0.071209,-0.056087,-0.00685,-0.088804,-0.053474,-0.097453,-0.021866,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.284952,0.229542,0.285276,0.229644,0.221899,0.156893,0.213572,0.173447,0.245159,0.184967,0.20007,0.169209,0.234142,0.179367,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167
64509,67,130,612,4914,2,17,325,5631,38,898,53,33,23,-0.092022,0.090638,-0.028767,-0.031928,-0.020947,-0.034987,-0.052131,-0.040579,-0.062163,0.296579,0.248604,0.583783,0.304347,-0.170566,-0.129027,-0.178954,-0.212705,-0.057546,-0.00685,0.00448,-0.068463,-0.097453,-0.023153,…,0.178327,0.219865,0.178327,0.21989,0.178322,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.219669,0.174029,0.284952,0.229542,0.285276,0.229644,0.221899,0.156893,0.213572,0.173447,0.1919,0.16194,0.20007,0.169209,0.210722,0.170611,0.219768,0.177842,0.216472,0.176785,0.219483,0.173818,0.215644,0.171729,0.218238,0.173167


In [13]:
train.write_csv('train.csv')
test.write_csv('test.csv')